In [ ]:
import os
import numpy as np
import scipy.sparse
import faiss
from multiprocessing.pool import ThreadPool
from faiss.contrib import clustering
from faiss.contrib.datasets import SyntheticDataset

This script demonstrates how to cluster vectors that are composed of a dense
part of dimension d1 and a sparse part of dimension d2 where d2 >> d1. 
The centroids are represented as full dense vectors. 

The implementation relies on the `clustering.DatasetAssign` object, that abstracts
away the representation of the vectors to cluster. The `clustering` module contains 
a pure Python implementation of `kmeans` that can consume this `DatasetAssign`. 

## Sparse-dense assignment class

In [16]:
def sparse_dense_assign_to_dense(xdense, xsparse, xb, xq_norms, xb_norms):
    """ assignment function for hstack(xdense, xsparse), xdense and xb are dense, 
    xsparse is a CSR matrix. It uses uses a matrix multiplication. 
    The squared norms can be provided if available.
    """
    nq, ddense = xdense.shape
    assert nq == xsparse.shape[0]
    dsparse = xsparse.shape[1]
    assert ddense + dsparse == xb.shape[1]
    nb = xb.shape[0]
    d2 = xb_norms - 2 * xdense @ xb[:, :ddense].T 
    d2 -= 2 * xsparse @ xb[:, ddense:].T
    I = d2.argmin(axis=1)
    D = d2.ravel()[I + np.arange(nq) * nb] + xq_norms.ravel()
    return D, I


def sparse_dense_assign_to_dense_blocks(
        xdense, xsparse, xb, xq_norms=None, xb_norms=None, qbs=16384, bbs=16384, nt=None): 
    """
    decomposes the sparse_assign_to_dense function into blocks to avoid a
    possible memory blow up. Can be run in multithreaded mode, because scipy's
    sparse-dense matrix multiplication is single-threaded.
    """
    nq = xdense.shape[0]
    assert nq == xsparse.shape[0]
    nb = xb.shape[0]
    assert xq_norms is not None
    # prepare result arrays
    D = np.empty(nq, dtype="float32")
    D.fill(np.inf)
    I = -np.ones(nq, dtype=int)

    if xb_norms is None:
        xb_norms = (xb ** 2).sum(1)

    def handle_query_block(i):
        xdense_block  = xdense[i : i + qbs]
        xsparse_block = xsparse[i : i + qbs]
        xq_norms_block = xq_norms[i : i + qbs]
        Iblock = I[i : i + qbs]
        Dblock = D[i : i + qbs]
        for j in range(0, nb, bbs):
            Di, Ii = sparse_dense_assign_to_dense(
                xdense_block, xsparse_block,
                xb[j : j + bbs],
                xq_norms=xq_norms_block,
                xb_norms=xb_norms[j : j + bbs],
            )
            if j == 0:
                Iblock[:] = Ii
                Dblock[:] = Di
            else:
                mask = Di < Dblock
                Iblock[mask] = Ii[mask] + j
                Dblock[mask] = Di[mask]

    if nt == 0 or nt == 1 or nq <= qbs:
        list(map(handle_query_block, range(0, nq, qbs)))
    else:
        pool = ThreadPool(nt)
        pool.map(handle_query_block, range(0, nq, qbs))
    
    return D, I 


In [ ]:
class DatasetAssignDenseSparse(clustering.DatasetAssign):
    """Wrapper for a matrix that is the concatenation of a dense and
    a sparse set of columns."""

    def __init__(self, xdense, xsparse):
        assert xsparse.__class__ == scipy.sparse.csr_matrix
        self.xsparse = xsparse
        self.xdense = xdense
        self.squared_norms = (xdense**2).sum(1)[:, None] + np.array(
            xsparse.power(2).sum(1)
        )

    def get_subset(self, indices):
        return np.hstack(
            (self.xdense[indices], np.array(self.xsparse[indices].todense()))
        )

    def count(self):
        return self.xdense.shape[0]

    def dim(self):
        return self.xdense.shape[1] + self.xsparse.shape[1]

    def perform_search(self, centroids):
        return sparse_dense_assign_to_dense_blocks(
            self.xdense, self.xsparse, centroids, xq_norms=self.squared_norms, nt=None
        )

    def assign_to(self, centroids, weights=None):
        D, I = self.perform_search(centroids)

        I = I.ravel()
        D = D.ravel()
        n = self.xdense.shape[0]
        if weights is None:
            weights = np.ones(n, dtype="float32")
        nc = len(centroids)
        m = scipy.sparse.csc_matrix((weights, I, np.arange(n + 1)), shape=(nc, n))
        sum_per_centroid = np.hstack((
            np.array(m @ self.xdense),
            np.array((m * self.xsparse).todense())
        ))
        return I, D, sum_per_centroid

## Test

In [18]:
# generate the dense data 
ds = SyntheticDataset(64, 100000, 0, 0, seed=123)
# generate the sparse data (higher dimensional)
ds2 = SyntheticDataset(1000, 100000, 0, 0, seed=234)

In [5]:
# dense part 
xdense = ds.get_train()

In [20]:
# sparse part: make sure that less than 1% of the data is non-zero
xsparse = ds2.get_train()
mask = (np.abs(xsparse) < 0.9997)
xsparse[mask] = 0
# xsparse = scipy.sparse.csr_matrix(xsparse[mask])

### reference run

Reference run where the sparse part is just treated as dense and concatenated to the dense part 

In [7]:
data = clustering.DatasetAssign(np.hstack((xdense, xsparse)))   # this is the normal dense DatesetAssign
clusters, iteration_stats = clustering.kmeans(100, data, niter=10, return_stats=True)

  Iteration 9 (2.18 s, search 2.16 s): objective=3.44424e+06 imbalance=1.050 nsplit=0


In [8]:
iteration_stats[-1]["obj"]

3444237.0

### Run with real sparse support

Use the `DatasetAssignDenseSparse` object to handle the dense + sparse clustering.

In [14]:
xsparse_csr = scipy.sparse.csr_matrix(xsparse)
data = DatasetAssignDenseSparse(xdense, xsparse_csr)
clusters, iteration_stats = clustering.kmeans(100, data, niter=10, return_stats=True)

  Iteration 9 (8.44 s, search 8.42 s): objective=3.44421e+06 imbalance=1.049 nsplit=0


In [15]:
iteration_stats[-1]["obj"]

3444213.0